<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

1. Загрузка и подготовка данных.
2. Обучение разных моделей. 
3. Выводы.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import re
import spacy
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv')
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [4]:
data.isnull().sum()

Unnamed: 0    0
text          0
toxic         0
dtype: int64

In [5]:
data['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

Проверка баланса классов.

In [6]:
data['toxic'].value_counts(normalize=True)

0    0.898388
1    0.101612
Name: toxic, dtype: float64

Явный дисбаланс классов.

## Обучение

In [7]:
def clean_text(text):
    cleaned = re.sub(r'[^А-Яа-яA-Za-z]', ' ', text).lower()
    return cleaned

#w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
#lemmatizer = nltk.stem.WordNetLemmatizer()
load_model = spacy.load('en_core_web_sm', disable = ['parser','ner'])

def lemmatize_and_clean(text):
    cleaned_text = clean_text(text)
    doc = load_model(cleaned_text)
    #return ' '.join([lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(cleaned_text)])
    return ' '.join([token.lemma_ for token in doc])

In [8]:
tqdm.pandas()
data['lemmatized_text'] = data['text'].progress_map(lemmatize_and_clean)
features_train, features_test, target_train, target_test = train_test_split(data['lemmatized_text'], data['toxic'], test_size=0.1)
data.head()

  0%|          | 0/159292 [00:00<?, ?it/s]

,Unnamed: 0,text,toxic,lemmatized_text
0,0,Explanation\nWhy the edits made under my usern...,0,explanation why the edit make under my usernam...
1,1,D'aww! He matches this background colour I'm s...,0,d aww he match this background colour I m se...
2,2,"Hey man, I'm really not trying to edit war. It...",0,hey man I m really not try to edit war it ...
3,3,"""\nMore\nI can't make any real suggestions on ...",0,more I can t make any real suggestion on im...
4,4,"You, sir, are my hero. Any chance you remember...",0,you sir be my hero any chance you rememb...


Преобразование текста в векторы признаков

In [9]:
vectorizer = TfidfVectorizer()

X_train_vectorized = vectorizer.fit_transform(features_train)
X_test_vectorized = vectorizer.transform(features_test)

**LogisticRegression**

In [10]:
model = LogisticRegression(C=10, class_weight='balanced')

scores_list = cross_val_score(estimator=model, 
                              X=X_train_vectorized, 
                              y=target_train, 
                              cv=5,  
                              scoring="f1")

val_score = scores_list.mean()
print("F1 score:", val_score)

/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

F1 score: 0.7726993006684721


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


**MultinomialNB**

In [11]:
from sklearn.metrics import f1_score
nb_model = MultinomialNB()

param_grid = {
    'alpha': [0.1, 1, 10],
    'fit_prior': [True, False]
}

grid_search = GridSearchCV(nb_model, param_grid, scoring='f1', cv=5)
grid_search.fit(X_train_vectorized, target_train)
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_
best_score = grid_search.best_score_
#y_pred = best_model.predict(X_test_vectorized)
#f1 = f1_score(target_test, y_pred)
print("F1 Score:", best_score)
print("Лучшие параметры:", best_params)


F1 Score: 0.6227892632303382
Лучшие параметры: {'alpha': 0.1, 'fit_prior': True}


В качестве лучшей модели выбираем LogisticRegression с гиперпараметром *C* равным 10.

In [12]:
model.fit(X_train_vectorized, target_train)
y_pred = model.predict(X_test_vectorized)
f1 = f1_score(target_test, y_pred)
print("F1 Score:", f1)

F1 Score: 0.7766657134686874


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Выводы

В ходе работы был проанализирован датасет. Был выявлен дисбаланс классов, который в дальнейшем был учтен при инициализациии моделей. Удовлетворительный результат F1-score показала модель **LogisticRegression** - 0,7766657134686874.